In [ ]:
import csv, re, os, sys, gpxpy, folium, requests
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np

dir_path = './'
in_dir_name = 'rawdata'
out_dir_name = 'result'
time_tolerance = 1000000

In [ ]:
def init_dir():
    dir_list = os.listdir(dir_path)
    is_init = False
    if in_dir_name not in dir_list:
        print('作業ディレクトリが存在しません')
        os.mkdir(dir_path + in_dir_name)
        print('make: /' + in_dir_name)
        is_init = True
    if out_dir_name not in dir_list:
        if not is_init:
            print('作業ディレクトリが存在しません')
        os.mkdir(dir_path + out_dir_name)
        print('make: /' + out_dir_name)
        is_init = True
    if is_init:
        print('初期化が完了しました')
        sys.exit(1)

In [ ]:
def get_valid_num(pronpt='', get_int=True, max=100000):
    num = 0
    while True:
        if pronpt != '':
            print(pronpt)
        num = input('>')
        try:
            num = float(num)
            if num > max - 1 or num < 0:
                print(f'0以上{max}未満の数を入力してください')
                continue
            else:
                break
        except ValueError:
            print('数字を入力してください')
            continue
    if get_int:
        return int(num)
    else:
        return num

In [ ]:
def format_weather_data():
    print('=' * 50)
    print('*** 気象表作成 ***')
    weather_datas = os.listdir(dir_path + in_dir_name)
    weather_datas = [file for file in weather_datas if file.endswith(".csv")]
    if len(weather_datas) == 0:
        print(in_dir_name + 'に .csv ファイルが存在しません')
        sys.exit(2)

    print('使用する気象データファイルを選択')
    for i in range(len(weather_datas)):
        print(f'{i}) {weather_datas[i]}')
    i_filename = weather_datas[get_valid_num(max=len(weather_datas))]
    i_filename = dir_path + in_dir_name + '/' + i_filename
    print()

    with open(i_filename, 'r', encoding='MS932') as csvfile:
        reader = csv.reader(csvfile)

        dl_date = ''
        raw_data = []

        for row_idx, row in enumerate(reader):
            if row_idx == 0:
                dl_date = row[0][row[0].find('：') + 1:]
            if row_idx > 1:
                raw_data.append(row)

    get_md = lambda date: date[date.find('/') + 1:]
    md_start = get_md(raw_data[5][0])  # Month/Day start ex: 3/17
    md_end = get_md(raw_data[-1][0])  # Month/Day end ex: 3/19
    y_start = raw_data[5][0][:raw_data[5][0].find('/')]  # year start
    reference_years = 0
    for row in raw_data:
        if row[0].find(md_start) != -1:
            reference_years += 1
    reference_days = int((len(raw_data) - 5) / reference_years)

    processed_data = {}
    station_data = {}
    invalid_di = []  # invalid date and item
    get_loc = lambda li: raw_data[0][li] + '県' + raw_data[1][li] + '地方'  # li: line_index
    loc = get_loc(1)
    for line_idx in range(1, len(raw_data[0])):
        if get_loc(line_idx) != loc:
            processed_data.setdefault(loc, station_data.copy())
            loc = get_loc(line_idx)
            station_data.clear()
        if raw_data[4][line_idx] == '':
            row = []
            for row_ix in range(5, len(raw_data)):
                if re.match(r'^[-+]?[0-9]*\.?[0-9]+$', raw_data[row_ix][line_idx]):
                    row.append(float(raw_data[row_ix][line_idx]))
                else:
                    row.append(0)
                    invalid_di.append('-'.join([raw_data[row_ix][0], loc, raw_data[2][line_idx]]))
            station_data.setdefault(raw_data[2][line_idx], row)
    processed_data.setdefault(loc, station_data.copy())

    must_items = ['平均気温(℃)', '最高気温(℃)', '最低気温(℃)', '降水量の合計(mm)']
    for mi in must_items:
        if mi not in station_data.keys():
            print(f'{i_filename} には観測項目【{mi}】が含まれていません\n'
                  f'気象データのダウンロードの設定を確認してください')
            sys.exit(2)
    if len(invalid_di) > 0:
        print(f'{i_filename} には欠損データが検出されました\n'
              f'これらのデータを 0 で置換しました\n'
              f'HA, AA, LA, LL の数値が適切に計算できません\n'
              f'検出個所：')
        for invalid in invalid_di:
            print('\t' + invalid)
    else:
        print(f'{i_filename} は適切に読み込まれました')
    print('-' * 50)

    site_infos = []  # site place name and elevation
    obs_infos = []  # observation point elevation
    defs = []

    keys_obs = list(processed_data.keys())  # keys of observation points
    for i in range(1, reference_days + 1):
        print(f'【{i}日目】\n観測地点を選択（半角数字）')
        for j, locs in enumerate(keys_obs):
            print(f'{j}) {locs}')
        obs_choice = get_valid_num(max=len(keys_obs))
        obs_point = keys_obs[obs_choice]
        obs_place_elev = get_valid_num(pronpt=f'{obs_point}の標高を入力', get_int=False)
        obs_infos.append((obs_point, obs_place_elev))

        print('サイト地の名称を入力（ENTERで省略）')
        site_place = input(' >')
        if site_place == '':
            site_place = str(i) + '日目サイト地'
        site_place_elev = get_valid_num(pronpt=f'{site_place}の標高を入力', get_int=False)
        site_infos.append((site_place, site_place_elev))
        defs.append((site_infos[i - 1][1] - obs_infos[i - 1][1]) * -0.006)
        print('-' * 50)

    print('保存ファイル名を入力（ENTERで省略）')
    o_filename = input('>')

    if '降水量の合計(mm)' in station_data:
        value_to_move = station_data['降水量の合計(mm)']
        del station_data['降水量の合計(mm)']
        station_data['降水量の合計(mm)'] = value_to_move

    write_data = []  # write to csv data
    write_data2 = [['', 'HH', 'HA', 'AA', 'LA', 'LL']]
    md_start = datetime.strptime(md_start, '%m/%d')  # convert str to date
    md_end = datetime.strptime(md_end, '%m/%d')
    d = date(datetime.now().year, md_start.month, md_start.day)
    if o_filename == '':
        o_filename = f'{dir_path}{out_dir_name}/{md_start.strftime("%m%d")}-{md_end.strftime("%m%d")}-weatherTable.csv'
    else:
        o_filename = f'{dir_path}{out_dir_name}/{o_filename}.csv'
    for i in range(reference_days):
        write_data.append([d.strftime('%Y/%m/%d') + ' DAY ' + str(i + 1)])
        write_data.append([f'観測地点：{obs_infos[i][0]} 標高：{obs_infos[i][1]}'])
        write_data.append([f'基準地点：{site_infos[i][0]} 標高：{site_infos[i][1]} 補正値：{defs[i]}K'])
        write_data.append([''] + [str(int(y_start) + j) for j in range(reference_years)])
        table = []
        table_crr = []
        sa = [''] * 5  # sum and average HH, HA, AA, LA, HA
        for item in station_data.keys():
            table.append(
                [item] + [processed_data.get(obs_infos[i][0]).get(item)[i + k * reference_days] for k in
                          range(reference_years)])
        for row in table:
            if '℃' in row[0]:
                table_crr = [round(row[elem_idx] + defs[i], 1) for elem_idx in range(1, len(row))]
                write_data.append([row[0]] + table_crr)
            else:
                write_data.append(row)
            if row[0] == '最高気温(℃)':
                sa[0] = str(max(table_crr[1:]))
                sa[1] = str(round(sum(table_crr[1:]) / reference_years, 1))
            elif row[0] == '平均気温(℃)':
                sa[2] = str(round(sum(table_crr[1:]) / reference_years, 1))
            elif row[0] == '最低気温(℃)':
                sa[4] = str(min(table_crr[1:]))
                sa[3] = str(round(sum(table_crr[1:]) / reference_years, 1))
        write_data2.append([d.strftime('%Y/%m/%d')] + sa)
        d += timedelta(days=1)
    write_data.append([])

    with open(o_filename, 'w', newline='', encoding='MS932') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(write_data)
        writer.writerows(write_data2)

In [ ]:
try:
    init_dir()

    format_weather_data()

    print('\n正常に終了しました')
    print('=' * 50)
except SystemExit:
    print('\nプログラムを終了します')

    print('=' * 50)